In [1]:
# import our necessary first packages
from bs4 import BeautifulSoup
import urllib
import pandas as pd
from datetime import datetime

In [121]:
import requests
from bs4 import BeautifulSoup
import urllib
import pandas as pd
from datetime import datetime

#collecting primary articles
stories = []
def getTheGoodStuff(newsstories):
    global stories
    global i
    for data in newsstories:
        htmlatag = data.find("h2", class_="title").find("a")
        headline = htmlatag.getText()
        url = htmlatag.get("href")
        d = {"headline" : headline,
             "url" : url,
            "sort_rank" : i}
        stories.append(d)
        for related in data.find_all(class_="related-item"):
            related_htmlatag = related.find("a")
            related_headline = related_htmlatag.getText()
            related_url = related_htmlatag.get("href")
            r = {"headline" : related_headline,
                 "url" : related_url,
                 "sort_rank" : i+1}
            stories.append(r)
            
def scrapeWebsites():
    global stories
    global i
    
    # Getting stories from Fox News.
    foxnews = "http://www.foxnews.com/"
    page = urllib.urlopen(foxnews)
    r  = requests.get(foxnews)
    data = r.text
    soup = BeautifulSoup(data,"lxml")
    for i in range(0, 20):
        foundstories = soup.find_all("article", class_="article story-" + str(i))
        getTheGoodStuff(foundstories)
    
scrapeWebsites()
articles = pd.DataFrame(stories)
articles.drop_duplicates(inplace=True)
articles['date'] = datetime.today().strftime('%Y-%m-%d')
articles['year'] = datetime.today().year
articles['month'] = datetime.today().month
articles['day'] = datetime.today().day
articles['hour'] = datetime.today().hour
articles['minute'] = datetime.today().minute

fox_domains = []
for web in articles.url:
    domain_items = web.split('//')[-1].split('/')[0].strip('www.').split('.')
    url_item = web.split('https://www.foxnews.com/')[-1].split('/')[0]
    if domain_items[0] in ['video','foxbusiness','nation']:
        fox_domains.append(domain_items[0])
    else:
        fox_domains.append(url_item)
articles['article_type'] = fox_domains

#collecting Opinion articles
sidebar_stories = []

def opinion_getTheGoodStuff(newsstories):
    global sidebar_stories
    for data in newsstories:
        htmlatag = data.find("h2", class_="title").find("a")
        headline = htmlatag.getText()
        url = htmlatag.get("href")
        d = {"headline" : headline,
             "url" : url}
        sidebar_stories.append(d)

def opinion_scrapeWebsites():
    global sidebar_stories
    foxnews = "http://www.foxnews.com/"
    page = urllib.urlopen(foxnews)
    r  = requests.get(foxnews)
    data = r.text
    soup = BeautifulSoup(data,"lxml")
    foundstories = soup.find(class_="collection collection-opinion has-load-more js-opinion").find_all("article")
    for article in foundstories:
        opinion_getTheGoodStuff(foundstories)
        
opinion_scrapeWebsites()
sidebar_articles = pd.DataFrame(sidebar_stories)
sidebar_articles.drop_duplicates(inplace=True)
sort_ranks = range(sidebar_articles.shape[0]+1)
del sort_ranks[0]
sidebar_articles["sort_rank"] = sort_ranks
sidebar_articles['date'] = datetime.today().strftime('%Y-%m-%d')
sidebar_articles['year'] = datetime.today().year
sidebar_articles['month'] = datetime.today().month
sidebar_articles['day'] = datetime.today().day
sidebar_articles['hour'] = datetime.today().hour
sidebar_articles['minute'] = datetime.today().minute

sidebar_articles['article_type'] = "opinion"

all_articles = pd.concat([articles,sidebar_articles],sort=False)

#harvesting article text
from selenium import webdriver
from time import sleep

article_text = []
driver = webdriver.Chrome("C:\Users\Duncan\Data Science\chromedriver.exe")

for url in all_articles.url:
    driver.get(url)
    sleep(1) #wait 1 second
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser', from_encoding="utf-8")
    article_text.append(soup.find_all('p'))

driver.close()    
all_articles['article_text'] = article_text

foxnews_articles = pd.read_csv('foxnews.csv')
print "foxnews articles current shape:"
print foxnews_articles.shape[0]
print ""
conc = pd.concat([foxnews_articles,all_articles],sort=False)
conc.reset_index(inplace=True)
conc.drop(['index','Unnamed: 0'],axis=1,inplace=True)

conc.to_csv('foxnews.csv', encoding = 'utf-8')
print "foxnews new shape:"
print conc.shape[0]

foxnews articles current shape:
83

foxnews new shape:
132


In [124]:
conc

,headline,sort_rank,url,date,year,month,day,hour,minute,article_type,article_text
0,"Hate-Trump media is just lies, noise",1,https://video.foxnews.com/v/6042498605001/,2019-05-29,2019,5,29,23,15,video,"[<p>Sign in to comment!</p>, <p>May. 29, 2019 ..."
1,Giuliani unleashes blistering attack on Muelle...,1,https://www.foxnews.com/politics/rudy-giuliani...,2019-05-29,2019,5,29,23,15,politics,"[<p class=""copyright"" data-optly-6dcb091b-a70c..."
2,DOJ watchdog: Former FBI official illegally le...,1,https://www.foxnews.com/politics/doj-watchdog-...,2019-05-29,2019,5,29,23,15,politics,"[<p class=""copyright"" data-optly-6dcb091b-a70c..."
3,Explore some of America's national treasures,1,https://nation.foxnews.com/watch/8ae7532aa938e...,2019-05-29,2019,5,29,23,15,nation,[]
4,CNN host faces criticism after strange comment...,1,https://www.foxnews.com/entertainment/cnn-host...,2019-05-29,2019,5,29,23,15,entertainment,"[<p class=""copyright"" data-optly-6dcb091b-a70c..."
5,New acting ICE director on the border crisis,2,https://video.foxnews.com/v/6042506733001/,2019-05-29,2019,5,29,23,15,video,"[<p>Sign in to comment!</p>, <p>May. 29, 2019 ..."
6,GERALDO RIVERA: Mueller dealt Pelosi two 'deuc...,2,https://www.foxnews.com/politics/geraldo-river...,2019-05-29,2019,5,29,23,15,politics,"[<p class=""copyright"" data-optly-6dcb091b-a70c..."
7,Canadian PM Trudeau plans to confront Pence on...,2,https://www.foxnews.com/politics/canadian-pm-t...,2019-05-29,2019,5,29,23,15,politics,"[<p class=""copyright"" data-optly-6dcb091b-a70c..."
8,2 small planes crash in mid-air near Florida a...,2,https://www.foxnews.com/us/florida-small-plane...,2019-05-29,2019,5,29,23,15,us,"[<p class=""copyright"" data-optly-6dcb091b-a70c..."
9,Mueller has nothing more to say,3,https://video.foxnews.com/v/6042492592001/,2019-05-29,2019,5,29,23,15,video,"[<p>Sign in to comment!</p>, <p>May. 29, 2019 ..."


In [112]:
soup.find("article", class_="article story-16").find("h2", class_="title").find("a")

<a href="https://www.foxnews.com/us/9-11-memorial-glade-honors-rescue-recovery-workers-affected-by-toxin-exposure">9/11 exhibit honors rescue, recovery workers affected by toxin exposure</a>

In [113]:
found_related_stories = soup.find_all("article", class_="article story-1")
for article in article1:
    print article.find_all(class_="related-item").

[]
[<li class="related-item"><a href="https://www.foxnews.com/politics/andrew-mccabe-mueller-trump-msnbc">ANDREW McCABE: Takeaway from Mueller presser is 'no witch hunt,' 'no exoneration'</a></li>, <li class="related-item"><a href="https://www.foxnews.com/entertainment/cnn-dom-lemon-trump-voters-brainwashed">CNN's Don Lemon attempts to shame Trump voters over Mueller: 'Are they brainwashed?'</a></li>, <li class="related-item"><a href="https://www.foxnews.com/entertainment/jim-carrey-robert-mueller-cowardly-lion-wizard-of-oz-artwork">Jim Carrey slams Robert Mueller as \u2018Cowardly Lion\u2019 in \u2018Wizard of Oz\u2019-inspired artwork</a></li>, <li class="related-item"><a href="https://www.foxnews.com/politics/rand-paul-justin-amash-mueller-probe">\nRand Paul on fellow libertarian Amash's impeachment call: Russia probe was 'un-libertarian'</a></li>, <li class="related-item"><a href="https://www.foxnews.com/politics/rudy-giuliani-dare-mueller-testify">RUDY GIULIANI: I'd love to see Mu

In [118]:
stories = []
def getTheGoodStuff(newsstories):
    global stories
    global i
    for data in newsstories:
        htmlatag = data.find("h2", class_="title").find("a")
        headline = htmlatag.getText()
        url = htmlatag.get("href")
        d = {"headline" : headline,
             "url" : url,
            "sort_rank" : i}
        stories.append(d)
        for related in data.find_all(class_="related-item"):
            related_htmlatag = related.find("a")
            related_headline = related_htmlatag.getText()
            related_url = related_htmlatag.get("href")
            r = {"headline" : related_headline,
                 "url" : related_url,
                 "sort_rank" : i+1}
            stories.append(r)
            
def scrapeWebsites():
    global stories
    global i
    
    # Getting stories from Fox News.
    foxnews = "http://www.foxnews.com/"
    page = urllib.urlopen(foxnews)
    r  = requests.get(foxnews)
    data = r.text
    soup = BeautifulSoup(data,"lxml")
    for i in range(0, 20):
        foundstories = soup.find_all("article", class_="article story-" + str(i))
        getTheGoodStuff(foundstories)
    
scrapeWebsites()
articles = pd.DataFrame(stories)
articles.drop_duplicates(inplace=True)
articles['date'] = datetime.today().strftime('%Y-%m-%d')
articles['year'] = datetime.today().year
articles['month'] = datetime.today().month
articles['day'] = datetime.today().day
articles['hour'] = datetime.today().hour
articles['minute'] = datetime.today().minute

fox_domains = []
for web in articles.url:
    domain_items = web.split('//')[-1].split('/')[0].strip('www.').split('.')
    url_item = web.split('https://www.foxnews.com/')[-1].split('/')[0]
    if domain_items[0] in ['video','foxbusiness','nation']:
        fox_domains.append(domain_items[0])
    else:
        fox_domains.append(url_item)
articles['article_type'] = fox_domains

#collecting Opinion articles
sidebar_stories = []

def opinion_getTheGoodStuff(newsstories):
    global sidebar_stories
    for data in newsstories:
        htmlatag = data.find("h2", class_="title").find("a")
        headline = htmlatag.getText()
        url = htmlatag.get("href")
        d = {"headline" : headline,
             "url" : url}
        sidebar_stories.append(d)

def opinion_scrapeWebsites():
    global sidebar_stories
    foxnews = "http://www.foxnews.com/"
    page = urllib.urlopen(foxnews)
    r  = requests.get(foxnews)
    data = r.text
    soup = BeautifulSoup(data,"lxml")
    foundstories = soup.find(class_="collection collection-opinion has-load-more js-opinion").find_all("article")
    for article in foundstories:
        opinion_getTheGoodStuff(foundstories)
        
opinion_scrapeWebsites()
sidebar_articles = pd.DataFrame(sidebar_stories)
sidebar_articles.drop_duplicates(inplace=True)
sort_ranks = range(sidebar_articles.shape[0]+1)
del sort_ranks[0]
sidebar_articles["sort_rank"] = sort_ranks
sidebar_articles['date'] = datetime.today().strftime('%Y-%m-%d')
sidebar_articles['year'] = datetime.today().year
sidebar_articles['month'] = datetime.today().month
sidebar_articles['day'] = datetime.today().day
sidebar_articles['hour'] = datetime.today().hour
sidebar_articles['minute'] = datetime.today().minute

sidebar_articles['article_type'] = "opinion"

all_articles = pd.concat([articles,sidebar_articles],sort=False)

#harvesting article text
from selenium import webdriver
from time import sleep

article_text = []
driver = webdriver.Chrome("C:\Users\Duncan\Data Science\chromedriver.exe")

for url in all_articles.url:
    driver.get(url)
    sleep(1) #wait 1 second
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser', from_encoding="utf-8")
    article_text.append(soup.find_all('p'))

driver.close()    
all_articles['article_text'] = article_text

In [119]:
all_articles

,headline,sort_rank,url,date,year,month,day,hour,minute,article_type,article_text
0,Hannity: Mueller contradicted himself during s...,1,https://video.foxnews.com/v/6042980294001/,2019-05-30,2019,5,30,22,29,video,"[<p>Sign in to comment!</p>, <p>May. 30, 2019 ..."
1,Dems' demands for testimony may trigger troubl...,1,https://www.foxnews.com/politics/robert-muelle...,2019-05-30,2019,5,30,22,29,politics,"[<p class=""copyright"" data-optly-6dcb091b-a70c..."
2,Barr says he 'personally felt' Mueller could h...,2,https://www.foxnews.com/politics/william-barr-...,2019-05-30,2019,5,30,22,29,politics,"[<p class=""copyright"" data-optly-6dcb091b-a70c..."
3,Jim Carrey slams Robert Mueller as ‘Cowardly L...,2,https://www.foxnews.com/entertainment/jim-carr...,2019-05-30,2019,5,30,22,29,entertainment,"[<p class=""copyright"" data-optly-6dcb091b-a70c..."
4,CNN's Don Lemon attempts to shame Trump voters...,2,https://www.foxnews.com/entertainment/cnn-dom-...,2019-05-30,2019,5,30,22,29,entertainment,"[<p class=""copyright"" data-optly-6dcb091b-a70c..."
5,Juan Williams: 'I don't think Mueller has a ri...,2,https://www.foxnews.com/politics/juan-williams...,2019-05-30,2019,5,30,22,29,politics,"[<p class=""copyright"" data-optly-6dcb091b-a70c..."
7,Trump announces escalating tariffs against Mex...,1,https://www.foxnews.com/politics/trump-mexico-...,2019-05-30,2019,5,30,22,29,politics,"[<p class=""copyright"" data-optly-6dcb091b-a70c..."
8,Watch Park'd and other shows featuring one of ...,1,https://nation.foxnews.com/watch/8ae7532aa938e...,2019-05-30,2019,5,30,22,29,nation,[]
9,"Dollar Tree renovating 1,000 Family Dollar sto...",1,https://www.foxbusiness.com/retail/dollar-tree...,2019-05-30,2019,5,30,22,29,foxbusiness,"[<p data-modulename=""launcher"">This website is..."
10,Tucker: Impeachment looks certain,2,https://video.foxnews.com/v/6042963668001/,2019-05-30,2019,5,30,22,29,video,"[<p>Sign in to comment!</p>, <p>May. 30, 2019 ..."


In [120]:
all_articles.article_text[40]

[<p class="copyright" data-optly-6dcb091b-a70c-4c25-b431-3878b10cee2e="">This material may not be published, broadcast, rewritten, or redistributed. \xa92019 FOX News Network, LLC. All rights reserved. All market data delayed 20 minutes.</p>,
 <p data-v-275a95fe="">Fox News Flash top headlines for May 30 are here. Check out what's clicking on Foxnews.com</p>,
 <p class="speakable">An <a href="https://www.foxnews.com/category/us/us-regions/southeast/alabama">Alabama</a> man who avoided execution last month for the <a href="https://www.foxnews.com/category/us/crime">stabbing death</a> of a preacher during a Christmastime robbery was put to death by lethal injection Thursday evening.</p>,
 <p class="speakable">The condemned man,\xa0<a href="https://www.foxnews.com/us/alabama-sets-new-execution-date-for-sword-and-dagger-slaying">Christopher Lee </a>Price, was\xa0pronounced dead at 7:31 p.m. at the\xa0William C. Holman Correctional Facility in Atmore. Price released a statement through his 

In [24]:
sidebar_articles.shape

(49, 8)

In [22]:
import requests
from bs4 import BeautifulSoup
import urllib

stories = []

def getTheGoodStuff(newsstories):
    global stories
    global i
    for data in newsstories:
        htmlatag = data.find("h2", class_="title").find("a")
        headline = htmlatag.getText()
        url = htmlatag.get("href")
        d = {"headline" : headline,
             "url" : url,
            "sort_rank" : i}
        stories.append(d)

def scrapeWebsites():
    global stories
    global i
    
    # Getting stories from Fox News.
    foxnews = "http://www.foxnews.com/"
    page = urllib.urlopen(foxnews)
    r  = requests.get(foxnews)
    data = r.text
    soup = BeautifulSoup(data,"lxml")
    for i in range(0, 20):
        foundstories = soup.find_all("article", class_="article story-" + str(i))
        getTheGoodStuff(foundstories)
    
#def displayStories():
#    global stories
#    global i
#    for i in range(0, len(stories)):
#        print stories[i]["headline"]
#        print stories[i]['url']
#        print stories[i]['i']
#        print ""
    
scrapeWebsites()
articles = pd.DataFrame(stories)
articles['date'] = datetime.today().strftime('%Y-%m-%d')
articles['year'] = datetime.today().year
articles['month'] = datetime.today().month
articles['day'] = datetime.today().day
articles['hour'] = datetime.today().hour
articles['minute'] = datetime.today().minute

fox_domains = []
for web in articles.url:
    domain_items = web.split('//')[-1].split('/')[0].strip('www.').split('.')
    url_item = web.split('https://www.foxnews.com/')[-1].split('/')[0]
    if domain_items[0] in ['video','foxbusiness','nation']:
        fox_domains.append(domain_items[0])
    else:
        fox_domains.append(url_item)
articles['article_type'] = fox_domains
articles

,headline,sort_rank,url,date,year,month,day,hour,minute,article_type
0,"Hannity: Hate-Trump media is just lies, noise",1,https://video.foxnews.com/v/6042498605001/,2019-05-29,2019,5,29,22,19,video
1,Giuliani unleashes blistering attack on Muelle...,1,https://www.foxnews.com/politics/rudy-giuliani...,2019-05-29,2019,5,29,22,19,politics
2,"Hannity: Hate-Trump media is just lies, noise",1,https://video.foxnews.com/v/6042498605001/,2019-05-29,2019,5,29,22,19,video
3,DOJ watchdog: Former FBI official illegally le...,1,https://www.foxnews.com/politics/doj-watchdog-...,2019-05-29,2019,5,29,22,19,politics
4,\nExplore some of America's national treasures,1,https://nation.foxnews.com/watch/8ae7532aa938e...,2019-05-29,2019,5,29,22,19,nation
5,CNN host faces criticism after strange comment...,1,https://www.foxnews.com/entertainment/cnn-host...,2019-05-29,2019,5,29,22,19,entertainment
6,New acting ICE director on the border crisis,2,https://video.foxnews.com/v/6042506733001/,2019-05-29,2019,5,29,22,19,video
7,GERALDO RIVERA: Mueller dealt Pelosi two 'deuc...,2,https://www.foxnews.com/politics/geraldo-river...,2019-05-29,2019,5,29,22,19,politics
8,New acting ICE director on the border crisis,2,https://video.foxnews.com/v/6042506733001/,2019-05-29,2019,5,29,22,19,video
9,"Uncle of missing Utah girl, 5, charged with mu...",2,https://www.foxnews.com/us/utah-girl-elizabeth...,2019-05-29,2019,5,29,22,19,us


In [38]:
soup.find_all("article"

[<article class="story no-author" data-social="1" data-v-54a96eec=""><!-- --><div class="inner" data-v-54a96eec=""><!-- --><!-- --><!-- --><header class="article-header" data-v-54a96eec=""><img alt="Fox Business" class="logo-printable ae-img" layout="responsive" src="//global.fncstatic.com/static/orion/styles/img/fox-business/logos/fb-header-wide.png" style="display: none;"/><h1>4 signs you should fire a freelance client</h1><div class="meta"><p><!-- --><!-- --><time class="date no-author" datetime="2010-07-31">Published May 26, 2019</time><span class="tags"><a href="/small-business">Small Business</a></span><span class="source"><a href="http://www.fool.com">Motley Fool</a></span></p></div></header><div class="social" data-v-54a96eec=""><ul><li class="fb"><a href="#"><span>Facebook</span></a></li><li class="tw"><a href="#"><span>Twitter</span></a></li><li class="comments"><a href="#"><span>Comments</span></a></li><li class="print"><a href="#"><span>Print</span></a></li></ul></div><div 

In [66]:
articles

,headline,sort_rank,url,date,year,month,day,hour,minute,article_type
0,Memorial Day ceremony at Arlington National Ce...,1,http://video.foxnews.com/v/6041597944001/,2019-05-27,2019,5,27,11,53,video
1,NYU professor apologizes over false tweet that...,1,https://www.foxnews.com/politics/trump-seizes-...,2019-05-27,2019,5,27,11,53,politics
2,Memorial Day ceremony at Arlington National Ce...,1,http://video.foxnews.com/v/6041597944001/,2019-05-27,2019,5,27,11,53,video
3,JOCKO WILLINK: You sacrificed your dreams for ...,1,https://www.foxnews.com/opinion/jocko-willink-...,2019-05-27,2019,5,27,11,53,opinion
4,Binge stories that celebrate our country and h...,1,https://nation.foxnews.com/watch/d0e61fa49443c...,2019-05-27,2019,5,27,11,53,nation
5,CBS star anchor says he was fired because he '...,1,https://www.foxnews.com/us/scott-pelley-reveal...,2019-05-27,2019,5,27,11,53,us
6,Wounded veteran receives new smart home,2,https://video.foxnews.com/v/6041597307001/,2019-05-27,2019,5,27,11,53,video
7,DANIEL TURNER: Stealth AOC 'Green New Deal' no...,2,https://www.foxnews.com/opinion/daniel-turner-...,2019-05-27,2019,5,27,11,53,opinion
8,Wounded veteran receives new smart home,2,https://video.foxnews.com/v/6041597307001/,2019-05-27,2019,5,27,11,53,video
9,"Homeless woman, 25, killed in ‘horrendous bash...",2,https://www.foxnews.com/world/homeless-woman-h...,2019-05-27,2019,5,27,11,53,world


In [44]:
articles.url[15]

'https://www.foxnews.com/real-estate/summer-moving-real-estate'

In [64]:
articles.domain.value_counts()

foxnews        23
video          16
foxbusiness     3
nation          1
Name: domain, dtype: int64

In [81]:
# set the url we want to visit
url = "https://www.foxnews.com/"

# visit that url, and grab the html of said page
html = urllib.urlopen(url).read()

soup = BeautifulSoup(html, 'html.parser', from_encoding="utf-8")

soup.find_all(class_='related')

[<div class="related"><ul><li class="related-item"><a href="https://www.foxnews.com/politics/andrew-mccabe-mueller-trump-msnbc">ANDREW McCABE: Takeaway from Mueller presser is 'no witch hunt,' 'no exoneration'</a></li><li class="related-item"><a href="https://www.foxnews.com/entertainment/cnn-dom-lemon-trump-voters-brainwashed">CNN's Don Lemon attempts to shame Trump voters over Mueller: 'Are they brainwashed?'</a></li><li class="related-item"><a href="https://www.foxnews.com/entertainment/jim-carrey-robert-mueller-cowardly-lion-wizard-of-oz-artwork">Jim Carrey slams Robert Mueller as \u2018Cowardly Lion\u2019 in \u2018Wizard of Oz\u2019-inspired artwork</a></li><li class="related-item"><a href="https://www.foxnews.com/politics/rand-paul-justin-amash-mueller-probe">\nRand Paul on fellow libertarian Amash's impeachment call: Russia probe was 'un-libertarian'</a></li><li class="related-item"><a href="https://www.foxnews.com/politics/rudy-giuliani-dare-mueller-testify">RUDY GIULIANI: I'd 

In [82]:
soup.prettify()

u'<!DOCTYPE html>\n<html lang="en">\n <head>\n  <meta charset="utf-8"/>\n  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>\n  <meta content="width=device-width,minimum-scale=1,initial-scale=1" name="viewport"/>\n  <meta content="homepage" name="pagetype">\n   <meta content="noarchive,noodp" name="robots"/>\n   <title>\n    Fox News - Breaking News Updates | Latest News Headlines | Photos &amp; News Videos\n   </title>\n   <meta content="Breaking News, Latest News and Current News from FOXNews.com. Breaking news and video. Latest Current News: U.S., World, Entertainment, Health, Business, Technology, Politics, Sports." name="description"/>\n   <meta content="news, breaking news, latest news, current news, world news, national news" name="keywords"/>\n   <link href="//static.foxnews.com/static/orion/styles/img/fox-news/favicons/favicon.ico" rel="shortcut icon" type="image/x-icon"/>\n   <link href="//static.foxnews.com/static/orion/styles/img/fox-news/favicons/apple-touch-icon-57x5

In [28]:
sidebar_stories = []

def opinion_getTheGoodStuff(newsstories):
    global sidebar_stories
    global record
    for data in newsstories:
        htmlatag = data.find("h2", class_="title").find("a")
        headline = htmlatag.getText()
        url = htmlatag.get("href")
        d = {"headline" : headline,
             "url" : url}
        sidebar_stories.append(d)

def opinion_scrapeWebsites():
    global sidebar_stories
    global record
    
    # Getting stories from Fox News.
    foxnews = "http://www.foxnews.com/"
    page = urllib.urlopen(foxnews)
    r  = requests.get(foxnews)
    data = r.text
    soup = BeautifulSoup(data,"lxml")
    foundstories = soup.find(class_="collection collection-opinion has-load-more js-opinion").find_all("article")
    for article in foundstories:
        opinion_getTheGoodStuff(foundstories)
        
opinion_scrapeWebsites()
sidebar_articles = pd.DataFrame(sidebar_stories)
sidebar_articles.drop_duplicates(inplace=True)

sidebar_articles['date'] = datetime.today().strftime('%Y-%m-%d')
sidebar_articles['year'] = datetime.today().year
sidebar_articles['month'] = datetime.today().month
sidebar_articles['day'] = datetime.today().day
sidebar_articles['hour'] = datetime.today().hour
sidebar_articles['minute'] = datetime.today().minute

sidebar_articles['article_type'] = "opinion"

sidebar_articles

,headline,url,date,year,month,day,hour,minute,article_type
0,Mueller makes impeachment tightrope tougher to...,https://www.foxnews.com/opinion/andrew-mccarth...,2019-05-29,2019,5,29,22,29,opinion
1,Gutfeld assesses the Democratic field,https://www.foxnews.com/opinion/gutfeld-assess...,2019-05-29,2019,5,29,22,29,opinion
2,"\nWhat about our American ranchers, President ...",https://www.foxnews.com/opinion/tomi-lahren-wh...,2019-05-29,2019,5,29,22,29,opinion
3,Is impeachment inevitable? Here's how Pelosi c...,https://www.foxnews.com/opinion/gary-meltz-rob...,2019-05-29,2019,5,29,22,29,opinion
4,Mueller has given Democrats a gift – and passe...,https://www.foxnews.com/opinion/leslie-marshal...,2019-05-29,2019,5,29,22,29,opinion
5,NYC's radical push for 'equity' in schools is ...,https://www.foxnews.com/opinion/deroy-murdock-...,2019-05-29,2019,5,29,22,29,opinion
6,\nThe real reason Obama intel officials don't ...,https://www.foxnews.com/opinion/tucker-carlson...,2019-05-29,2019,5,29,22,29,opinion


In [14]:
len(soup.find(class_="collection collection-opinion has-load-more js-opinion").find_all("article"))

7

In [107]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width,minimum-scale=1,initial-scale=1" name="viewport"/>
  <meta content="homepage" name="pagetype">
   <meta content="noarchive,noodp" name="robots"/>
   <title>
    Fox News - Breaking News Updates | Latest News Headlines | Photos &amp; News Videos
   </title>
   <meta content="Breaking News, Latest News and Current News from FOXNews.com. Breaking news and video. Latest Current News: U.S., World, Entertainment, Health, Business, Technology, Politics, Sports." name="description"/>
   <meta content="news, breaking news, latest news, current news, world news, national news" name="keywords"/>
   <link href="//static.foxnews.com/static/orion/styles/img/fox-news/favicons/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
   <link href="//static.foxnews.com/static/orion/styles/img/fox-news/favicons/apple-touch-icon-57x57.png" rel="appl

In [10]:
soup.find(class_="collection collection-opinion has-load-more js-opinion").find_all("article")

[<article class="article"><div class="info"><header class="info-header"><h2 class="title"><a href="https://www.foxnews.com/opinion/andrew-mccarthy-muellers-press">Mueller makes impeachment tightrope tougher to walk for Dems</a></h2></header><div class="meta"><span class="author">By\xa0<a href="//www.foxnews.com/person/m/andrew-mccarthy.html">Andrew McCarthy</a></span></div></div><div class="m headshot"><a href="//www.foxnews.com/person/m/andrew-mccarthy.html"><img alt="Andrew McCarthy" data-src="//a57.foxnews.com/hp.foxnews.com/images/2019/05/128/128/18bdf6c0a975e6d85b2843a05c9cffcc.jpg" src="//global.fncstatic.com/static/orion/img/clear.gif"/></a></div></article>,
 <article class="article"><div class="info"><header class="info-header"><h2 class="title"><a href="https://www.foxnews.com/opinion/gutfeld-assesses-the-democratic-field">Gutfeld assesses the Democratic field</a></h2></header><div class="meta"><span class="author">By\xa0<a href="//www.foxnews.com/person/g/greg-gutfeld.html">G

In [153]:
soup.find_all(True, {"class":["collection collection-opinion has-load-more js-opinion","article"]})#"collection collection-opinion has-load-more js-opinion")
#opnion_soup = BeautifulSoup(opinion, 'html.parser', from_encoding="utf-8")

[<article class="article story-1"><div class="m"><a data-video-id="6042058272001" href="https://video.foxnews.com/v/6042058272001/"><img alt="Sen. Sanders holds a rally in Manchester, NH" src="https://a57.foxnews.com/media2.foxnews.com/BrightCove/694940094001/2019/05/28/480/270/694940094001_6042054622001_6042058272001-vs.jpg"/></a><span class="pill is-livenow">Live Now</span><span class="pill duration"><a class="pill-link" href="https://video.foxnews.com/v/6042058272001/">Live</a></span></div><div class="info"><header class="info-header"><div class="meta"></div><h2 class="title"><a href="https://video.foxnews.com/v/6042058272001/">Sen. Sanders holds a rally in Manchester, NH</a></h2></header></div></article>,
 <article class="article story-2"><div class="m"><a data-video-id="6042002186001" href="https://video.foxnews.com/v/6042002186001/"><img alt="Sanders, Bennet make the rounds in New Hampshire; Buttigieg offers access; Biden target California donors" src="https://a57.foxnews.com/med

In [161]:
opinion = BeautifulSoup(soup.find(class_="collection collection-opinion has-load-more js-opinion"), 'html.parser', from_encoding="utf-8")

TypeError: 'NoneType' object is not callable

In [5]:
# set the url we want to visit
url = "https://www.foxnews.com/"

# visit that url, and grab the html of said page
html = urllib.urlopen(url).read()

# we need to convert this into a soup object
soup = BeautifulSoup(html, 'html.parser', from_encoding="utf-8")

In [69]:
soup.find_all('p')

AttributeError: ResultSet object has no attribute 'getText'. You're probably treating a list of items like a single item. Did you call find_all() when you meant to call find()?